In [ ]:
!pip install transformers biopython torch

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from Bio import SeqIO
import numpy as np
from tqdm import tqdm
import os
import gc

# --- CẤU HÌNH CHO MODEL 650M ---
MODEL_NAME = "facebook/esm2_t33_650M_UR50D"
TRAIN_FASTA = "/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta"
TEST_FASTA = "/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta"

# Với Model 650M trên GPU T4 (16GB VRAM):
# Batch size = 4-6 là an toàn nhất. Nếu crash thì giảm xuống 2.
BATCH_SIZE = 6 
MAX_CONTEXT = 1024 
EMBED_DIM = 1280  # Model 650M tạo ra vector 1280 chiều

In [ ]:
# --- 1. KHỞI TẠO ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Device: {device}")

print(f"Loading {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

# Bắt buộc dùng FP16 để chạy được Batch Size hợp lý
if device.type == 'cuda':
    model = model.half()
    
model.to(device)
model.eval()

In [ ]:
# --- 2. HÀM TẠO EMBEDDING (Hỗ trợ chuỗi dài) ---
def generate_embeddings_650M(fasta_path, prefix_name):
    print(f"\n--- Processing {prefix_name.upper()} set with ESM2-650M ---")
    
    if not os.path.exists(fasta_path):
        print(f"⚠️ Warning: File {fasta_path} not found.")
        return

    # Load dữ liệu
    ids = []
    seqs = []
    for record in SeqIO.parse(fasta_path, "fasta"):
        ids.append(record.id)
        seqs.append(str(record.seq))
    
    print(f"Found {len(seqs)} sequences.")
    
    # Sắp xếp theo độ dài để tối ưu tốc độ (giảm padding thừa)
    sorted_indices = np.argsort([len(s) for s in seqs])
    seqs_sorted = [seqs[i] for i in sorted_indices]
    
    batch_embeddings_list = []
    
    # Bắt đầu vòng lặp
    for i in tqdm(range(0, len(seqs_sorted), BATCH_SIZE), desc=f"Embedding {prefix_name}"):
        batch_seqs = seqs_sorted[i : i + BATCH_SIZE]
        
        # Kiểm tra xem có chuỗi nào dài quá 1022 ký tự không
        has_long_seq = any(len(s) > (MAX_CONTEXT - 2) for s in batch_seqs)
        
        if has_long_seq:
            # --- CHIẾN LƯỢC CHUNKING (Cắt nhỏ & Gộp) ---
            # Dành cho protein dài để không bị mất thông tin
            for seq in batch_seqs:
                chunks = [seq[j : j + (MAX_CONTEXT-2)] for j in range(0, len(seq), (MAX_CONTEXT-2))]
                chunk_vectors = []
                
                for chunk in chunks:
                    inputs = tokenizer(chunk, return_tensors="pt", padding=False, truncation=True, max_length=MAX_CONTEXT).to(device)
                    with torch.no_grad():
                        out = model(**inputs).last_hidden_state
                        # Mean pooling (bỏ CLS/EOS)
                        if out.shape[1] > 2: vec = out[0, 1:-1, :].mean(dim=0)
                        else: vec = out[0, :, :].mean(dim=0)
                        chunk_vectors.append(vec.float().cpu().numpy())
                
                # Tính trung bình các đoạn
                batch_embeddings_list.append(np.mean(chunk_vectors, axis=0))
                
        else:
            # --- CHIẾN LƯỢC BATCH NHANH ---
            inputs = tokenizer(batch_seqs, return_tensors="pt", padding=True, truncation=True, max_length=MAX_CONTEXT).to(device)
            with torch.no_grad():
                outputs = model(**inputs)
                last_hidden_state = outputs.last_hidden_state
                attention_mask = inputs['attention_mask']
                
                # Masking chuẩn
                mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
                if device.type == 'cuda': mask_expanded = mask_expanded.half()

                sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)
                sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
                
                # Kết quả batch
                batch_embs = (sum_embeddings / sum_mask).float().cpu().numpy()
                batch_embeddings_list.extend(batch_embs)
        
        # Clear VRAM thường xuyên hơn vì model 650M khá nặng
        if i % (BATCH_SIZE * 20) == 0: 
            torch.cuda.empty_cache()

    # Sắp xếp lại đúng thứ tự ban đầu
    final_embeddings = np.zeros((len(seqs), EMBED_DIM), dtype=np.float32)
    for idx, original_idx in enumerate(sorted_indices):
        final_embeddings[original_idx] = batch_embeddings_list[idx]
    
    # Lưu file
    # Đặt tên suffix _650M để dễ phân biệt
    np.save(f"{prefix_name}_embeddings_650M.npy", final_embeddings)
    np.save(f"{prefix_name}_ids.npy", np.array(ids))
    print(f"✅ Saved: {prefix_name}_embeddings_650M.npy ({final_embeddings.shape})")
    
    # Giải phóng RAM triệt để
    del final_embeddings, batch_embeddings_list, seqs, ids
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
# --- 3. CHẠY ---
generate_embeddings_650M(TRAIN_FASTA, "train")
generate_embeddings_650M(TEST_FASTA, "test")

print("\n🎉 DONE 650M PIPELINE!")